In [1]:
!pip install langchain==0.3.19
!pip install langchain-openai==0.3.6
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00


In [2]:
from getpass import getpass
OPENAI_KEY = getpass('please enter your Open AI API KEY HERE ....:')

please enter your Open AI API KEY HERE ....:··········


In [3]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [4]:
from langchain_openai import ChatOpenAI
chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


In [6]:
# Define your desired data structure - like a python data class
class QueryResponse(BaseModel):
  description: str = Field(description="A brief description of the topic asked by user")
  pros: str = Field(description="3 bullet points showing the pros of the topic asked by user")
  cons: str = Field(description="3 bullet points showing the cons of the topic asked by user")
  conclusion: str = Field(description="One line conclusion of the topic asked by the user")
parser = PydanticOutputParser(pydantic_object=QueryResponse)
parser


PydanticOutputParser(pydantic_object=<class '__main__.QueryResponse'>)

In [7]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "A brief description of the topic asked by user", "title": "Description", "type": "string"}, "pros": {"description": "3 bullet points showing the pros of the topic asked by user", "title": "Pros", "type": "string"}, "cons": {"description": "3 bullet points showing the cons of the topic asked by user", "title": "Cons", "type": "string"}, "conclusion": {"description": "One line conclusion of the topic asked by the user", "title": "Conclusion", "type": "string"}}, "required": ["description", "pros", 

In [8]:
prompt_txt = """
Answer the user query and generate the response based on the following formatting instructions
Format Instruction:
{format_instructions}

Query:
{query}
"""

prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"description": "A brief description of the topic asked by user", "title": "Description", "type": "string"}, "pros": {"description": "3 bullet points showing the pros of the topic asked by user", "title": "Pros", "type": "string"}, "cons": {"description": "3 bullet points showing the cons of the topic asked by user", "title": "Cons", "type": "string"}, "conclusion": {"description": "One line conclusion of t

In [9]:
chain = (prompt | chatgpt | parser)

In [14]:
#question ="Tell me about Credit Card Features"
question ="Explain me about the significant of Pydantic method in Langchain, explain in details"
response = chain.invoke({"query": question})
response

QueryResponse(description='Pydantic is a data validation and settings management library for Python that plays a significant role in Langchain by ensuring that the data structures used in the framework are validated and correctly typed, enhancing reliability and maintainability.', pros='- Provides data validation, ensuring that the data conforms to expected types and formats.\n- Enhances code readability and maintainability by using Python type hints.\n- Facilitates easier debugging and error handling by raising clear validation errors.', cons="- May introduce additional overhead in terms of performance due to validation checks.\n- Requires developers to learn and understand Pydantic's features and syntax.\n- Can lead to complexity in cases where dynamic data structures are frequently changing.", conclusion='Overall, Pydantic significantly enhances data integrity and developer experience in Langchain.')

In [11]:
response.description

'Credit cards are financial tools that allow users to borrow funds for purchases, with the obligation to pay back the borrowed amount along with interest and fees.'

In [12]:
response.dict()

<ipython-input-12-83cee9ab0c35>:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response.dict()


{'description': 'Credit cards are financial tools that allow users to borrow funds for purchases, with the obligation to pay back the borrowed amount along with interest and fees.',
 'pros': '1. Convenient for making purchases without carrying cash.\n2. Offers rewards and cashback on spending.\n3. Helps build credit history and improve credit score.',
 'cons': '1. High-interest rates can lead to debt if not managed properly.\n2. Potential for overspending due to easy access to credit.\n3. Fees for late payments and cash advances can accumulate quickly.',
 'conclusion': 'Credit cards can be beneficial if used responsibly, but they require careful management to avoid debt.'}

In [15]:
for k, v in response.dict().items():
  print(f'{k}:\n{v}\n')

description:
Pydantic is a data validation and settings management library for Python that plays a significant role in Langchain by ensuring that the data structures used in the framework are validated and correctly typed, enhancing reliability and maintainability.

pros:
- Provides data validation, ensuring that the data conforms to expected types and formats.
- Enhances code readability and maintainability by using Python type hints.
- Facilitates easier debugging and error handling by raising clear validation errors.

cons:
- May introduce additional overhead in terms of performance due to validation checks.
- Requires developers to learn and understand Pydantic's features and syntax.
- Can lead to complexity in cases where dynamic data structures are frequently changing.

conclusion:
Overall, Pydantic significantly enhances data integrity and developer experience in Langchain.



<ipython-input-15-797b2bb3d6bb>:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  for k, v in response.dict().items():


# JsonOutput Parser

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List

# Define your desired data structure - like a python data class
class QueryResponse(BaseModel):
  description: str = Field(description="A brief description of the topic asked by user")
  pros: str = Field(description="3 bullet points showing the pros of the topic asked by user")
  cons: str = Field(description="3 bullet points showing the cons of the topic asked by user")
  conclusion: str = Field(description="One line conclusion of the topic asked by the user")
parser = JsonOutputParser(pydantic_object=QueryResponse)
parser

JsonOutputParser(pydantic_object=<class '__main__.QueryResponse'>)

In [17]:
prompt_txt = """
Answer the user query and generate the response based on the following formatting instructions

Format Instruction:
{format_instructions}

Query:
{query}
"""

prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"description": {"description": "A brief description of the topic asked by user", "title": "Description", "type": "string"}, "pros": {"description": "3 bullet points showing the pros of the topic asked by user", "title": "Pros", "type": "string"}, "cons": {"description": "3 bullet points showing the cons of the topic asked by user", "title": "Cons", "type": "string"}, "conclusion": {"description": "One line conclusion of t

In [18]:
chain = (prompt | chatgpt | parser)

In [19]:
topic_queries = ["Tell me about the HDFC Bank", "Tell me about ICICI Bank",
                 "Tell me about Generative AI", "Tell me about LogisticRegression Model"]

topic_queries_formated =[{"query":topic} for topic in topic_queries]
topic_queries_formated

[{'query': 'Tell me about the HDFC Bank'},
 {'query': 'Tell me about ICICI Bank'},
 {'query': 'Tell me about Generative AI'},
 {'query': 'Tell me about LogisticRegression Model'}]

In [20]:
response = chain.map().invoke(topic_queries_formated)
response

[{'description': "HDFC Bank is one of India's leading private sector banks, offering a wide range of financial services and products to its customers.",
  'pros': '1. Strong financial performance and stability.\n2. Extensive branch and ATM network across India.\n3. Innovative digital banking solutions and services.',
  'cons': '1. High competition in the banking sector may affect growth.\n2. Customer service issues reported by some users.\n3. Limited international presence compared to some global banks.',
  'conclusion': 'HDFC Bank is a robust choice for banking needs in India, with both strengths and challenges.'},
 {'description': "ICICI Bank is one of India's largest private sector banks, offering a wide range of financial services and products to customers.",
  'pros': '1. Extensive branch and ATM network across India.\n2. Diverse range of financial products including loans, insurance, and investment options.\n3. Strong digital banking platform for convenient online transactions.',

In [22]:
for response in response:
  for k, v in response.items():
    print(f"{k}:\n{v}\n")
    print("***************************")

description:
HDFC Bank is one of India's leading private sector banks, offering a wide range of financial services and products to its customers.

***************************
pros:
1. Strong financial performance and stability.
2. Extensive branch and ATM network across India.
3. Innovative digital banking solutions and services.

***************************
cons:
1. High competition in the banking sector may affect growth.
2. Customer service issues reported by some users.
3. Limited international presence compared to some global banks.

***************************
conclusion:
HDFC Bank is a robust choice for banking needs in India, with both strengths and challenges.

***************************
description:
ICICI Bank is one of India's largest private sector banks, offering a wide range of financial services and products to customers.

***************************
pros:
1. Extensive branch and ATM network across India.
2. Diverse range of financial products including loans, insurance

In [21]:
import pandas as pd
df = pd.DataFrame(response)
df

,description,pros,cons,conclusion
0,HDFC Bank is one of India's leading private se...,1. Strong financial performance and stability....,1. High competition in the banking sector may ...,HDFC Bank is a robust choice for banking needs...
1,ICICI Bank is one of India's largest private s...,1. Extensive branch and ATM network across Ind...,"1. Customer service can be inconsistent, leadi...","Overall, ICICI Bank is a prominent player in t..."
2,Generative AI refers to algorithms that can ge...,- Can create unique and diverse content quickl...,- May produce biased or inappropriate content ...,Generative AI holds great potential but requir...
3,Logistic Regression is a statistical method us...,1. Simple to implement and interpret.\n2. Work...,1. Assumes a linear relationship between the f...,Logistic Regression is a powerful tool for bin...


# Mini-Project (Real Time)

1. Review Analysis : Objective (Summarize, Highlight the pos and neg, Sentiment analysis, display the customer emotion)

2. Research Paper Analysis

3. Social Media Marketing Analysis

4. IT Support Analysis

In [38]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

In [24]:
reviews

["\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    ",
 "\n    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.\n    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,\n    enhancing my gaming experience significantly. Price-wise, it's quite competitive,\n    and I feel like I got a good deal. The delivery was swift, and it came well-protected,\n    ensuring no damage during transport.\n    ",
 "\n    Ordered a set of wireless earbuds for running, and they've been a letdown.\n    The sound constantl

In [39]:
# Define your desired data structure - like a python data class
class ReviewAnalysisResponse(BaseModel):
  summary: str = Field(description="A brief summary of the customer review with maximum 3 lines.")
  positive: str = Field(description="A list showing the positives mentioned by the customer in the review if any - max 3 points")
  negative: str = Field(description="A list showing the negative mentioned by the customer in the review if any - max 3 points")
  sentiment: str = Field(description="One word showing the sentiment of the review - Positive, Negative or Neutral")
  emotion: str=Field(description="A list of 3-5 emotions expressed by the customer in the review")
  email: str=Field(description="Detailed email to the customer based on the sentiment")
parser = PydanticOutputParser(pydantic_object=ReviewAnalysisResponse)
parser


PydanticOutputParser(pydantic_object=<class '__main__.ReviewAnalysisResponse'>)

In [40]:
prompt_txt = """
Analyse the given customer review below and generate the response based on the instructions
mentioned below in the format instructions.
Also remember to write a detailed email response for the email field based on these conditions:
- email should be addressed to Dear Customer and signed with Server Agent
- thank them if the review is positive or neutral
- apologize if the review is negative

Format Instructions:
{format_instructions}

Review:
{review}
"""

prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["review"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)
prompt

PromptTemplate(input_variables=['review'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"summary": {"description": "A brief summary of the customer review with maximum 3 lines.", "title": "Summary", "type": "string"}, "positive": {"description": "A list showing the positives mentioned by the customer in the review if any - max 3 points", "title": "Positive", "type": "string"}, "negative": {"description": "A list showing the negative mentioned by the customer in the review if any - max 3 points", "title": "N

In [41]:
# Create a LCEL LLM Chain
chain = (prompt | chatgpt | parser)

In [42]:
# Format the input review
reviews_formatted = [{'review': review} for review in reviews]
reviews_formatted

[{'review': "\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    "},
 {'review': "\n    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.\n    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,\n    enhancing my gaming experience significantly. Price-wise, it's quite competitive,\n    and I feel like I got a good deal. The delivery was swift, and it came well-protected,\n    ensuring no damage during transport.\n    "},
 {'review': "\n    Ordered a set of wireless earbuds for running, and they've been 

In [43]:
# Get response from the LLM
response = chain.map().invoke(reviews_formatted)
for response in response:
  for k, v in response.dict().items():
    print(f"{k}:\n{v}\n")
    print("***************************")
    print('\n')

summary:
The customer is highly satisfied with the Bluetooth speaker, praising its sound quality, waterproof feature, and compact design. They also appreciated the early shipping and secure packaging.

***************************


positive:
Sound quality is clear with good bass; Waterproof feature works well; Compact design with high volume.

***************************


negative:


***************************


sentiment:
Positive

***************************


emotion:
Happy, Satisfied, Impressed

***************************


email:
Dear Customer,

Thank you for your wonderful review! We're thrilled to hear that you are enjoying your new Bluetooth speaker and that it met your expectations in terms of sound quality, waterproof features, and compact design. It's great to know that the early shipping and secure packaging added to your positive experience.

If you have any further questions or need assistance, feel free to reach out. We appreciate your support and hope you have many f

<ipython-input-43-2f15849e8922>:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  for k, v in response.dict().items():


In [50]:
responses = chain.map().invoke(reviews_formatted)
responses

[ReviewAnalysisResponse(summary='The customer is highly satisfied with the Bluetooth speaker, praising its sound quality, waterproof feature, and compact design. They also appreciated the early shipping and secure packaging.', positive='Sound quality is clear with good bass; Waterproof feature works well; Compact design with high volume.', negative='', sentiment='Positive', emotion='Happy, Satisfied, Impressed', email="Dear Customer,\n\nThank you for your wonderful review! We're thrilled to hear that you are enjoying your new Bluetooth speaker and that it met your expectations in sound quality, waterproof capability, and design. It's great to know that the early shipping and secure packaging added to your positive experience.\n\nIf you have any further questions or need assistance, feel free to reach out. We appreciate your support and hope you have many great beach outings with your speaker!\n\nBest regards,\nServer Agent"),
 ReviewAnalysisResponse(summary='The customer is highly sati

In [53]:
import pandas as pd

df = pd.DataFrame(response.dict() for response in responses)
df.to_csv("df1.csv")


<ipython-input-53-57c7db628e1a>:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df = pd.DataFrame(response.dict() for response in responses)


In [46]:
response.dict()

<ipython-input-46-83cee9ab0c35>:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  response.dict()


{'summary': 'The customer is dissatisfied with the kitchen blender, citing performance issues and delivery delays. They expected better quality from the brand.',
 'positive': '',
 'negative': "Struggles with tough foods, incredibly noisy, ineffective 'easy-clean' feature, arrived late.",
 'sentiment': 'Negative',
 'emotion': 'Frustration, disappointment, dissatisfaction, regret',
 'email': 'Dear Customer,\n\nThank you for taking the time to share your feedback regarding the kitchen blender. We sincerely apologize for the issues you have experienced with the product, including its performance with tougher foods, the noise level, and the ineffective cleaning feature. We also regret the delay in delivery, which is not the standard we strive for.\n\nYour experience is important to us, and we would like to make it right. Please feel free to reach out to our customer service team for assistance or to discuss possible solutions.\n\nThank you for your understanding.\n\nBest regards,\nServer Ag

In [54]:
df

,summary,positive,negative,sentiment,emotion,email
0,The customer is highly satisfied with the Blue...,Sound quality is clear with good bass; Waterpr...,,Positive,"Happy, Satisfied, Impressed","Dear Customer,\n\nThank you for your wonderful..."
1,The customer is highly satisfied with their ne...,"Responsive keys, vibrant LED colors, competiti...",,Positive,"Satisfaction, Excitement, Contentment","Dear Customer,\n\nThank you for your wonderful..."
2,The customer is disappointed with the wireless...,They arrived on time.,"Sound cuts out, uncomfortable fit, poor batter...",Negative,"Disappointment, Frustration, Dissatisfaction","Dear Customer,\n\nThank you for taking the tim..."
3,The customer is disappointed with the tablet s...,The product arrived promptly.,"The stand wobbles easily, angles do not hold a...",Negative,"Disappointment, Frustration, Dissatisfaction","Dear Customer,\n\nThank you for your feedback ..."
4,The customer is dissatisfied with the kitchen ...,,"Struggles with tough foods, incredibly noisy, ...",Negative,"Frustration, disappointment, dissatisfaction, ...","Dear Customer,\n\nThank you for taking the tim..."
